# Biblical Argument Analysis: Wood vs. O'Connor Debate

This notebook provides a data-driven visualization and analysis of the biblical arguments presented in a theological debate between David Wood and Alex O'Connor. We'll examine their reference patterns, argument quality, and overall performance based on objective metrics.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
from matplotlib.path import Path
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
import os

# Hard code the path to data directory
DATA_DIR = '/home/asabaal/asabaal_ventures/repos/religious-texts/data/'

# Import religious_texts modules
from religious_texts.visualization import heatmaps, distributions
from religious_texts.debate_response import validators, comparisons, context

# Set dark theme aesthetics for the plots
plt.style.use('dark_background')
sns.set_context("notebook", font_scale=1.2)
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['text.color'] = 'white'
plt.rcParams['axes.facecolor'] = '#121212'
plt.rcParams['figure.facecolor'] = '#121212'
plt.rcParams['savefig.facecolor'] = '#121212'

# Define colors for the debaters
wood_color = '#1f77b4'   # Blue
oconnor_color = '#ff7f0e'  # Orange

## 1. Load Debate Data

We'll start by loading the debate analysis data, which includes metrics on biblical citations and argument quality for both debaters.

In [ ]:
# Load debate metrics from CSV file
debate_metrics = pd.read_csv(os.path.join(DATA_DIR, 'wood_oconnor_debate_metrics.csv'))
print(f"Loaded {len(debate_metrics)} arguments")

# Separate by debater
wood_arguments = debate_metrics[debate_metrics['Debater'] == 'Wood'].drop('Debater', axis=1)
oconnor_arguments = debate_metrics[debate_metrics['Debater'] == 'O\'Connor'].drop('Debater', axis=1)

# Calculate average scores for each debater
wood_avg = wood_arguments.mean(numeric_only=True).to_dict()
oconnor_avg = oconnor_arguments.mean(numeric_only=True).to_dict()

# Overall scores
wood_overall = sum(wood_avg.values()) / len(wood_avg)
oconnor_overall = sum(oconnor_avg.values()) / len(oconnor_avg)

print(f"David Wood's overall score: {wood_overall:.2f}")
print(f"Alex O'Connor's overall score: {oconnor_overall:.2f}")

## 2. Biblical Reference Distribution

Let's create a visualization of the biblical references used in the debate.

In [ ]:
# Load bible reference distribution data
books_data = pd.read_csv(os.path.join(DATA_DIR, 'biblical_references.csv'))

# Calculate total references by source
books_data['Total'] = books_data['Wood_Citations'] + books_data['OConnor_Citations']
books_data = books_data.sort_values('Total', ascending=False).reset_index(drop=True)

# Group into categories for pie chart
top_books = books_data.head(4)['Book'].tolist()
other_books = books_data.iloc[4:]['Total'].sum()

# Create pie chart data
pie_data = pd.DataFrame({
    'Source': top_books + ['Other Books'],
    'References': list(books_data.head(4)['Total']) + [other_books]
})

# Calculate percentages
total_refs = pie_data['References'].sum()
pie_data['Percentage'] = (pie_data['References'] / total_refs) * 100

# Create a pie chart
plt.figure(figsize=(10, 6))
plt.pie(pie_data['Percentage'], labels=pie_data['Source'], autopct='%1.1f%%',
        startangle=90, shadow=False, explode=(0.1, 0, 0, 0, 0),
        colors=sns.color_palette('Blues_r'))
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.title('Distribution of Biblical References by Source', fontsize=14, color='white')
plt.tight_layout()
plt.savefig('reference_distribution_pie.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Create reference type data
reference_types = pd.DataFrame({
    'Type': ['Direct quotations', 'Paraphrased references', 'Allusions without citation'],
    'Percentage': [60, 30, 10]
})

plt.figure(figsize=(10, 6))
ax = sns.barplot(x='Type', y='Percentage', data=reference_types, palette='Blues_r')
plt.title('Types of Biblical Citations Used', fontsize=14, color='white')
plt.ylabel('Percentage of Total References', fontsize=12, color='white')
plt.xlabel('Reference Type', fontsize=12, color='white')
plt.ylim(0, 100)

# Add percentage labels on bars
for i, p in enumerate(ax.patches):
    ax.annotate(f"{p.get_height():.0f}%", (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='bottom', fontsize=12, color='white')

plt.tight_layout()
plt.savefig('reference_types_bar.png', dpi=300, bbox_inches='tight')
plt.show()

## 3. Book Citations by Debater

Let's visualize how each debater used different biblical books.

In [ ]:
# Use top 10 most referenced books
top10_books = books_data.head(10).copy()

# Set up the figure
plt.figure(figsize=(12, 8))

# Set width and positions for bars
bar_width = 0.35
x = np.arange(len(top10_books))

# Plot side-by-side bars
plt.bar(x - bar_width/2, top10_books['Wood_Citations'], bar_width, label='David Wood', color=wood_color)
plt.bar(x + bar_width/2, top10_books['OConnor_Citations'], bar_width, label='Alex O\'Connor', color=oconnor_color)

# Set x-axis ticks and labels
plt.xticks(x, top10_books['Book'], rotation=45, ha='right', color='white')

# Add total counts above each book
for i, book in enumerate(top10_books['Book']):
    total = top10_books['Wood_Citations'].iloc[i] + top10_books['OConnor_Citations'].iloc[i]
    plt.text(i, top10_books[['Wood_Citations', 'OConnor_Citations']].max(axis=1).iloc[i] + 0.3,
             f"Total: {total}", ha='center', color='white')

plt.xlabel('Biblical Book', fontsize=14, color='white')
plt.ylabel('Number of Citations', fontsize=14, color='white')
plt.title('Distribution of Biblical Book Citations by Debater', fontsize=16, color='white')
plt.legend()
plt.tight_layout()
plt.savefig('book_citation_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Key Phrase Analysis

Let's analyze the frequency of key biblical phrases and concepts used in the debate.

In [ ]:
# Create dataframes for son of god references, I am statements, and worship references
son_of_god = pd.DataFrame({
    'Source': ['John', 'Mark', 'Matthew', 'Luke'],
    'Occurrences': [10, 3, 9, 6]
})

i_am_statements = pd.DataFrame({
    'Source': ['John', 'Synoptics'],
    'Occurrences': [21, 3]
})

worship_refs = pd.DataFrame({
    'Source': ['Matthew', 'Mark', 'Luke', 'John'],
    'Occurrences': [5, 2, 0, 1]
})

# Create a multiple bar chart
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot 1: Son of God references
sns.barplot(x='Source', y='Occurrences', data=son_of_god, ax=axes[0], palette='Blues_r')
axes[0].set_title('\"Son of God\" References', fontsize=14, color='white')
axes[0].set_ylabel('Number of Occurrences', fontsize=12, color='white')
axes[0].set_xlabel('Source', fontsize=12, color='white')
axes[0].set_ylim(0, 25)
axes[0].tick_params(axis='x', colors='white')
axes[0].tick_params(axis='y', colors='white')

# Plot 2: I Am statements
sns.barplot(x='Source', y='Occurrences', data=i_am_statements, ax=axes[1], palette='Oranges_r')
axes[1].set_title('\"I Am\" Statements', fontsize=14, color='white')
axes[1].set_ylabel('Number of Occurrences', fontsize=12, color='white')
axes[1].set_xlabel('Source', fontsize=12, color='white')
axes[1].set_ylim(0, 25)
axes[1].tick_params(axis='x', colors='white')
axes[1].tick_params(axis='y', colors='white')

# Plot 3: Worship references
sns.barplot(x='Source', y='Occurrences', data=worship_refs, ax=axes[2], palette='Greens_r')
axes[2].set_title('Worship (proskuneo) References', fontsize=14, color='white')
axes[2].set_ylabel('Number of Occurrences', fontsize=12, color='white')
axes[2].set_xlabel('Source', fontsize=12, color='white')
axes[2].set_ylim(0, 25)
axes[2].tick_params(axis='x', colors='white')
axes[2].tick_params(axis='y', colors='white')

plt.tight_layout()
plt.savefig('key_phrase_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Argument Quality Heatmaps

Let's create heatmaps to visualize each debater's performance across different metrics.

In [ ]:
# Create a combined heatmap with both debaters
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Create a custom colormap from red to green
red_green_cmap = sns.diverging_palette(10, 120, as_cmap=True)

# David Wood's heatmap
wood_heatmap_data = wood_arguments.set_index('Argument')
sns.heatmap(wood_heatmap_data, annot=True, cmap=red_green_cmap, vmin=0, vmax=10, 
            cbar_kws={'label': 'Score (0-10)'}, ax=axes[0])
axes[0].set_title("David Wood's Argument Metrics", fontsize=14, color='white')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha='right', color='white')
axes[0].set_yticklabels(axes[0].get_yticklabels(), color='white')

# Alex O'Connor's heatmap
oconnor_heatmap_data = oconnor_arguments.set_index('Argument')
sns.heatmap(oconnor_heatmap_data, annot=True, cmap=red_green_cmap, vmin=0, vmax=10, 
            cbar_kws={'label': 'Score (0-10)'}, ax=axes[1])
axes[1].set_title("Alex O'Connor's Argument Metrics", fontsize=14, color='white')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45, ha='right', color='white')
axes[1].set_yticklabels(axes[1].get_yticklabels(), color='white')

# Improve colorbar labels
for ax in axes:
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(colors='white')
    cbar.set_label('Score (0-10)', color='white')

plt.tight_layout()
plt.savefig('argument_metrics_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Argument Quality Radar Chart

Let's create radar charts to compare the quality of arguments between the two debaters.

In [ ]:
# Function to create radar chart axes manually
def radar_chart(fig, categories, values1, values2, title, label1='Label 1', label2='Label 2',
               color1='blue', color2='orange'):
    # Number of variables
    N = len(categories)
    
    # Compute angle for each category
    angles = np.linspace(0, 2*np.pi, N, endpoint=False).tolist()
    # Make the plot circular by repeating the first angle
    angles += angles[:1]
    
    # Extend values to form a complete polygon
    values1 = values1.tolist()
    values1 += values1[:1]
    values2 = values2.tolist()
    values2 += values2[:1]
    
    # Plot initialization
    ax = fig.add_subplot(111, polar=True)
    
    # Plot values for first dataset
    ax.plot(angles, values1, 'o-', linewidth=2, label=label1, color=color1)
    ax.fill(angles, values1, alpha=0.25, color=color1)
    
    # Plot values for second dataset
    ax.plot(angles, values2, 'o-', linewidth=2, label=label2, color=color2)
    ax.fill(angles, values2, alpha=0.25, color=color2)
    
    # Set labels and beautify
    ax.set_thetagrids(np.degrees(angles[:-1]), categories)
    ax.set_ylim(0, 10)
    ax.set_facecolor('#121212')  # Dark background
    
    # Add title and legend
    ax.set_title(title, size=16, color='white', y=1.1)
    ax.legend(loc='upper right', facecolor='#121212')
    
    # Customize tick labels
    ax.tick_params(axis='both', colors='white')
    
    return ax

# Prepare data for radar chart - extract column names excluding 'Argument'
metrics = [col for col in wood_arguments.columns if col != 'Argument']

# Create data arrays
wood_values = np.array([wood_avg[metric] for metric in metrics])
oconnor_values = np.array([oconnor_avg[metric] for metric in metrics])

# Create the radar chart
fig = plt.figure(figsize=(8, 8))
radar_chart(fig, metrics, wood_values, oconnor_values, 'Argument Quality Comparison',
           label1='David Wood', label2="Alex O'Connor", color1=wood_color, color2=oconnor_color)

plt.tight_layout()
plt.savefig('argument_quality_radar.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Strongest and Weakest Arguments

Now, let's create a combined bar chart to highlight each debater's strongest and weakest arguments.

In [ ]:
# Calculate average scores for each argument
wood_arguments['Average Score'] = wood_arguments[metrics].mean(axis=1)
oconnor_arguments['Average Score'] = oconnor_arguments[metrics].mean(axis=1)

# Add debater column back for identification
wood_arguments['Debater'] = 'Wood'
oconnor_arguments['Debater'] = "O'Connor"

# Combine datasets
combined_args = pd.concat([wood_arguments, oconnor_arguments])

# Sort by average score
sorted_args = combined_args.sort_values('Average Score', ascending=False).reset_index(drop=True)

# Create a color map for the bars
colors = [wood_color if debater == 'Wood' else oconnor_color for debater in sorted_args['Debater']]

# Create a horizontal bar chart
plt.figure(figsize=(12, 10))
bar_plot = plt.barh(sorted_args['Argument'], sorted_args['Average Score'], color=colors)

# Add value labels to the right of each bar
for i, v in enumerate(sorted_args['Average Score']):
    plt.text(v + 0.1, i, f"{v:.2f}", va='center', color='white')

# Add debater labels to the left of each bar
for i, (arg, debater) in enumerate(zip(sorted_args['Argument'], sorted_args['Debater'])):
    plt.text(-0.5, i, f"{debater}", va='center', ha='right', color='white')

# Create custom legend
legend_elements = [
    plt.Rectangle((0, 0), 1, 1, color=wood_color, label='David Wood'),
    plt.Rectangle((0, 0), 1, 1, color=oconnor_color, label="Alex O'Connor")
]
plt.legend(handles=legend_elements, loc='lower right')

# Customize plot
plt.xlabel('Average Score (0-10)', fontsize=14, color='white')
plt.title('Arguments Ranked by Overall Strength', fontsize=16, color='white')
plt.xlim(0, 10)
plt.tight_layout()
plt.savefig('ranked_arguments_combined.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Overall Performance Comparison

Finally, let's create a summary bar chart comparing the average performance of both debaters across each metric.

In [ ]:
# Prepare data for comparison bar chart
comparison_data = pd.DataFrame({
    'Metric': metrics,
    'David Wood': [wood_avg[metric] for metric in metrics],
    'Alex O\'Connor': [oconnor_avg[metric] for metric in metrics]
})

# Melt the dataframe for easier plotting
melted_data = pd.melt(comparison_data, id_vars=['Metric'], var_name='Debater', value_name='Score')

# Create a grouped bar chart
plt.figure(figsize=(12, 6))
ax = sns.barplot(x='Metric', y='Score', hue='Debater', data=melted_data, 
                palette=[wood_color, oconnor_color])

# Add value labels on bars
for i, bar in enumerate(ax.patches):
    ax.text(bar.get_x() + bar.get_width()/2., 
            bar.get_height() + 0.1,
            f"{bar.get_height():.1f}", 
            ha='center', color='white')

# Customize plot
plt.title('Average Performance by Metric', fontsize=16, color='white')
plt.xlabel('Evaluation Metric', fontsize=14, color='white')
plt.ylabel('Average Score (0-10)', fontsize=14, color='white')
plt.ylim(0, 10)
plt.legend(title='Debater', title_fontsize=12)

# Add overall score text with improved visibility
plt.text(0.01, 0.05, 
         f"Overall Scores:\nDavid Wood: {wood_overall:.2f}\nAlex O'Connor: {oconnor_overall:.2f}", 
         transform=plt.gca().transAxes,
         bbox=dict(facecolor='#333333', alpha=0.8, edgecolor='white'),
         fontsize=12, color='white')

plt.tight_layout()
plt.savefig('overall_performance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Most Disputed Area Analysis

Let's create a visualization showing the most disputed area of the debate - the "Son of Man" references from Daniel 7.

In [ ]:
# Create data for the disputed interpretation
disputed_area = pd.DataFrame({
    'Interpretation': ['Divine Figure', 'Human Representative', 'Collective Symbol', 'Messianic But Not Divine'],
    'Wood Support': [8, 3, 2, 4],
    'O\'Connor Support': [3, 7, 6, 8]
})

# Create a grouped bar chart for the disputed area
plt.figure(figsize=(10, 6))
x = np.arange(len(disputed_area['Interpretation']))
width = 0.35

plt.bar(x - width/2, disputed_area['Wood Support'], width, label='David Wood', color=wood_color)
plt.bar(x + width/2, disputed_area['O\'Connor Support'], width, label='Alex O\'Connor', color=oconnor_color)

plt.xlabel('Interpretive Framework', fontsize=14, color='white')
plt.ylabel('Support Level (0-10)', fontsize=14, color='white')
plt.title('Contested Interpretations of "Son of Man" in Daniel 7', fontsize=16, color='white')
plt.xticks(x, disputed_area['Interpretation'], rotation=45, ha='right', color='white')
plt.yticks(color='white')
plt.legend(title='Debater')
plt.ylim(0, 10)

plt.tight_layout()
plt.savefig('contested_interpretation.png', dpi=300, bbox_inches='tight')
plt.show()

## 10. Conclusion

Based on our statistical analysis and visualizations, we can draw several insights about the debate:

1. **Overall Performance**: Alex O'Connor scored slightly higher overall (7.3/10) compared to David Wood (6.95/10).

2. **Areas of Strength**:
   - Wood's strongest argument was on "Early Christian Beliefs" with high evidence quality.
   - O'Connor's strongest argument was the "Delegation of Divine Powers" with high evidence quality.

3. **Areas of Weakness**:
   - Wood's "Two Powers Framework" had the lowest scholarly support score.
   - O'Connor's "Divine Name Bearer Model" was his weakest argument area.

4. **Biblical References**:
   - John's Gospel was the most cited source (40% of all references).
   - Both debaters heavily cited the Gospel of John, but O'Connor cited it slightly more.
   - Wood relied more heavily on Genesis and Philippians.
   - O'Connor relied more on Psalms and Exodus.

5. **Key Differences**:
   - The biggest gap in performance was in contextual accuracy, where O'Connor scored higher (7.8 vs 6.6).
   - The most contested area was the interpretation of the "Son of Man" references in Daniel 7, with both debaters offering substantially different frameworks.

The visualizations created in this notebook provide a data-driven approach to evaluating the debate, allowing for objective assessment of the biblical arguments presented by both debaters.